<a href="https://colab.research.google.com/github/hwnjoroge/R_vs_Python_Cheatsheet/blob/master/Introduction_to_Machine_Learning_Project_Hellen_Njoroge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

>**QUESTION**: 
HR analytics is revolutionising the way human resources departments operate, leadingto higher efficiency and better results overall. 
For my client, a large Multinational Corporation,the company needs my help in identifying the eligible candidates at a particular checkpoint and predicting whether a potential promotee at a checkpoint will be promoted or not after the evaluation processs.This will expedite the entire promotion cycle. 

>Multiple attributes around employees’ past and current performance along with demographics have been provided. ( ID for employee ,Department of employee,Region of employment,Employee Education Level,Gender of Employee,Channel of recruitment for employee,Number of other trainings completed in previous year on soft skills, technical skills etc, Age of Employee, Employee Rating for the previous year, Length of service in years,KPIs_met >80% ,Awards won during previous year, Average score in current training evaluations)

## Downloading the data, Analyzing the data


Importing required libraries

In [213]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#The following settings will improve the default style and font sizes for the charts in this notebook.
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')


load the dataset 

In [214]:
hr_df = pd.read_csv('https://bit.ly/2ODZvLCHRDataset')

In [215]:
hr_df

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54803,3030,Technology,region_14,Bachelor's,m,sourcing,1,48,3.0,17,0,0,78,0
54804,74592,Operations,region_27,Master's & above,f,other,1,37,2.0,6,0,0,56,0
54805,13918,Analytics,region_1,Bachelor's,m,other,1,27,5.0,3,1,0,79,0
54806,13614,Sales & Marketing,region_9,NaN,m,sourcing,1,29,1.0,2,0,0,45,0


The dataset contains 54808 rows and 14 columns. 
Our objective is to find a way to predict the value in the "is_promoted" column using the values in the other columns.


In [286]:
# Checking if there are duplicated employee records in the dataset using the employee_id column
duplicate = hr_df[hr_df.duplicated('employee_id')].sum()
duplicate

employee_id             0.0
department              0.0
region                  0.0
gender                  0.0
recruitment_channel     0.0
no_of_trainings         0.0
age                     0.0
previous_year_rating    0.0
length_of_service       0.0
KPIs_met >80%           0.0
awards_won?             0.0
avg_training_score      0.0
is_promoted             0.0
dtype: float64

Observations made
1.   Each record of the dataset represents one employee. there are no duplicated records using the employee_id, which is a unique value.
Therefore the dataset contains 54808 employee records and 14 columns describing various employee attributes. 


Checking the data types and missing values in the various columns.

In [216]:
#Checking the data types and missing values in the various columns.
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


The dataset contains numeric columns below - 
*   employee_id
*   no_of_trainings
*   age
*   previous_year_rating
*   length_of_service
*   KPIs_met >80%
*   awards_won
*   avg_training_score
*  is_promoted


while the below columns are strings (possibly categories). 
*   department
*   region
*   education
*   gender
*   recruitment_channel

Two of the columns contain missing values
1.   education
2.   previous_year_rating  




In [217]:
#checking how many rows are blank for the two columns 'education' and 'previous_year _rating'
hr_df.isna().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

Previous_year_rating has 4124 missing values and this was not dropped, as this might change the outcome of the data since over 4124 records would be deleted.

'Education' column was dropped 


In [218]:
hr_df =hr_df.drop(columns=['education'])

In [219]:
# Display the blank rows in the previous year rating column
empty_rating_rows = hr_df[hr_df.previous_year_rating.isnull()]
empty_rating_rows

,employee_id,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
10,29934,Technology,region_23,m,sourcing,1,30,NaN,1,0,0,77,0
23,71177,Procurement,region_5,m,other,1,27,NaN,1,0,0,70,0
29,74759,Sales & Marketing,region_4,m,sourcing,1,26,NaN,1,0,0,44,0
56,45709,Sales & Marketing,region_31,f,other,1,29,NaN,1,0,0,49,0
58,26599,Sales & Marketing,region_16,m,other,2,27,NaN,1,1,0,47,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54703,74615,R&D,region_31,m,sourcing,1,30,NaN,1,1,0,88,0
54734,11685,Operations,region_15,m,sourcing,1,31,NaN,1,1,0,56,1
54746,10546,Finance,region_6,m,other,1,28,NaN,1,1,0,61,0
54773,37919,Finance,region_2,m,other,1,23,NaN,1,1,0,61,0


Checking the statisticsof each of the hr dataset columns

In [220]:
hr_df.describe()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
count,54808.000000,54808.000000,54808.000000,50684.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000
mean,39195.830627,1.253011,34.803915,3.329256,5.865512,0.351974,0.023172,63.386750,0.085170
std,22586.581449,0.609264,7.660169,1.259993,4.265094,0.477590,0.150450,13.371559,0.279137
min,1.000000,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,39.000000,0.000000
25%,19669.750000,1.000000,29.000000,3.000000,3.000000,0.000000,0.000000,51.000000,0.000000
50%,39225.500000,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000,0.000000
75%,58730.500000,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,76.000000,0.000000
max,78298.000000,10.000000,60.000000,5.000000,37.000000,1.000000,1.000000,99.000000,1.000000


Some of the observations made from the dataset 
1.   the mean age of the data is 34, minimum is 20 while the max age is 60
2.   the longest serving employee in the dataset is 37
3.   the average trainign score achieved was 63 while the maximum was 99. 






In [221]:
# Checking if there are duplicated employee records in the dataset using the employee_id column
duplicate = hr_df[hr_df.duplicated('employee_id')].sum()
duplicate

employee_id             0.0
department              0.0
region                  0.0
gender                  0.0
recruitment_channel     0.0
no_of_trainings         0.0
age                     0.0
previous_year_rating    0.0
length_of_service       0.0
KPIs_met >80%           0.0
awards_won?             0.0
avg_training_score      0.0
is_promoted             0.0
dtype: float64

Observations made
1.   Each record of the dataset represents one employee. there are no duplicated records using the employee_id, which is a unique value












## Exploratory Data Analysis and Visualization

Exploring the distributions of various columns to see how they are related to the target column. 

Exploring and visualizing the data using the Plotly, Matplotlib and Seaborn libraries. 

In [222]:
# view how the promotions are distributed in the departments. sorted by descending order
px.histogram(hr_df, x='department', title='department vs. Promotion', color='is_promoted').update_xaxes(categoryorder='total descending')

Observations of the histogram : Department vs Promotion

Most promotions are from the sales and marketing department at 1213, then the Operations departments at 1013, and the least promotions from the legal department at 53.

Most of the employees who did not make it to be promoted also came from the sales department 15,627 while the least is from the R&D dept at 930

there is a possibility that this is because marketing department has the highest % of employees relative to other depratments. therefore more vacancies for promotions.

In [223]:
# view how the promotions are distributed against Age
px.histogram(hr_df, 
             x='age', 
             title='Age vs. Promotion', 
             color='is_promoted')

Observations drawn from the Age vs Promotion

The values of the ages seem to form a normal distribution centered around the values 30- 34, with a few outliers towards the right.

Highest count of the employees who did not get a promotion were aged 30 at 3341, similarly the highest count of employees who got a promotion were aged 30 (324)





In [224]:
# view how the promotions are distributed in the Regions. sorted by descending order
px.histogram(hr_df, x='region', title='Region vs. Promotion', color='is_promoted').update_xaxes(categoryorder='total descending')

Observations drawn from the Region vs Promotion

Most promotions are from the region_2 at 989, then the region_22 at 734, and the least promotions from region_18

Most of the employees who did not make it to be promoted also came from region_2 11354 while the least is from region_18



In [225]:
# view how the promotions are distributed by Gender
px.histogram(hr_df, x='gender', title='Gender vs. Promotion', color='is_promoted').update_xaxes(categoryorder='total descending')

Observations drawn from the Gender vs Promotion


Most of the promoted employees were male


In [226]:
# view how the promotions are distributed relative to the length of service
px.histogram(hr_df, x='length_of_service', marginal='box',  title='Length of Service vs. Promotion', color='is_promoted')

Observations drawn from the Length of service vs Promotion

Most of the employees in the company had a length of service of 2-5 years with a few outliers to the right.

similarly, most of the employees who were promoted and those who were not promoted, have been with the company for about 2-5 years. 


In [227]:
# view how the promotions are distributed relative to the Training Score
px.histogram(hr_df, x='avg_training_score', marginal='box', title='Training Score vs. Promotion', color='is_promoted')

Observations drawn from the Average Training Score vs Promotion

promoted employees perfomed better than the non promoted employees in the training, eg, 50% of the promoted employees scored 71, while 50%  of the non promoted employees scored 60. 



In [228]:
# view how the promotions are distributed relative to the Trainings completed , 
px.histogram(hr_df, x='no_of_trainings', title='Trainings vs. Promotion', color='is_promoted')

Observations drawn from the Number of trainings completed vs Promotion

Most of the emloyees completed only one training, Majority of the employees who completed one training did not get promoted.

## Training, Validation and Test Sets

Splitting the Dataset 

We split our dataset into training, validation and test sets. 60% of the data for the training set, 20% for the validation set and 20% for the test set.

In [229]:
 train_val_df, test_df = train_test_split(hr_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [230]:
print('hr_df.shape :', hr_df.shape)
print('train_df.shape :', train_df.shape)
print('val_df.shape :', val_df.shape)
print('test_df.shape :', test_df.shape)
print('train_val_df.shape :', train_val_df.shape)

hr_df.shape : (54808, 13)
train_df.shape : (32884, 13)
val_df.shape : (10962, 13)
test_df.shape : (10962, 13)
train_val_df.shape : (43846, 13)


In [231]:
train_df

,employee_id,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
50563,58212,Finance,region_11,m,other,1,27,4.0,4,1,0,55,0
31423,68964,Sales & Marketing,region_25,m,sourcing,1,35,3.0,10,1,0,52,0
50189,13163,Operations,region_22,m,other,1,47,3.0,5,0,0,70,1
23603,72150,Technology,region_7,f,other,1,42,4.0,3,1,0,82,0
25759,34112,Procurement,region_16,f,other,1,28,3.0,5,1,0,68,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48824,42713,Technology,region_24,f,sourcing,1,30,3.0,6,0,0,82,0
50281,70145,Operations,region_13,m,sourcing,1,41,1.0,7,0,0,58,0
6587,73307,Operations,region_27,m,other,1,27,4.0,3,0,0,64,0
51986,41418,Technology,region_13,f,sourcing,1,31,3.0,5,0,0,84,0


In [232]:
val_df

,employee_id,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
32148,22808,Legal,region_32,m,other,1,31,NaN,1,0,0,59,0
33302,45717,Procurement,region_22,m,sourcing,1,29,3.0,3,0,0,70,0
38582,17399,Technology,region_13,m,sourcing,1,32,3.0,6,1,0,82,0
26817,16809,Sales & Marketing,region_22,m,other,4,27,NaN,1,0,0,52,0
50418,71119,Finance,region_4,m,other,1,35,4.0,5,0,0,61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46589,49121,Technology,region_26,m,other,1,28,1.0,4,0,0,80,0
42762,50943,Analytics,region_31,m,other,1,29,3.0,6,0,0,88,0
8200,21284,Sales & Marketing,region_2,f,other,1,56,3.0,15,0,0,50,0
1391,17408,Operations,region_2,f,sourcing,1,36,4.0,2,1,0,59,0


In [233]:
test_df

,employee_id,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
40907,5599,HR,region_16,m,other,1,35,5.0,10,0,0,47,0
51939,73176,Sales & Marketing,region_7,m,other,3,33,1.0,4,0,0,51,0
6978,13157,Finance,region_2,m,other,2,38,3.0,11,0,0,64,0
12814,7458,Operations,region_22,f,other,1,33,1.0,8,0,0,56,0
22921,28446,Technology,region_7,f,other,1,37,1.0,4,0,0,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,78287,Analytics,region_7,m,other,1,35,3.0,2,0,0,83,0
26056,5601,Sales & Marketing,region_2,m,sourcing,2,50,5.0,3,1,0,45,0
29981,20907,Sales & Marketing,region_23,m,other,1,39,5.0,4,1,0,50,0
44352,24987,Sales & Marketing,region_7,f,other,1,31,2.0,5,0,0,49,0


## Identifying Input and Target Columns

In the current dataset, the employee_id column doea not have data needed for making the predictions. therefore it was excluded in the input column.
also, the target column 'is promoted' was identified and omitted from the training data

In [234]:
input_cols = list(train_df.columns)[1:-1]
target_col = 'is_promoted'

In [235]:
print(input_cols)

['department', 'region', 'gender', 'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score']


In [236]:
target_col

'is_promoted'

We can now create inputs and targets for the training, validation and test sets for further processing and model training.

In [237]:
train_inputs = train_df[input_cols].copy() # this is a dataframe
train_targets = train_df[target_col].copy() # this is a series

In [238]:
train_inputs

,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
50563,Finance,region_11,m,other,1,27,4.0,4,1,0,55
31423,Sales & Marketing,region_25,m,sourcing,1,35,3.0,10,1,0,52
50189,Operations,region_22,m,other,1,47,3.0,5,0,0,70
23603,Technology,region_7,f,other,1,42,4.0,3,1,0,82
25759,Procurement,region_16,f,other,1,28,3.0,5,1,0,68
...,...,...,...,...,...,...,...,...,...,...,...
48824,Technology,region_24,f,sourcing,1,30,3.0,6,0,0,82
50281,Operations,region_13,m,sourcing,1,41,1.0,7,0,0,58
6587,Operations,region_27,m,other,1,27,4.0,3,0,0,64
51986,Technology,region_13,f,sourcing,1,31,3.0,5,0,0,84


In [239]:
train_targets

50563    0
31423    0
50189    1
23603    0
25759    0
        ..
48824    0
50281    0
6587     0
51986    0
24101    0
Name: is_promoted, Length: 32884, dtype: int64

In [240]:
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

In [241]:
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

Identifying which of the columns are numerical and which ones are categorical. This will be useful later, as we'll need to convert the categorical data to numbers for training a logistic regression model.

In [242]:
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [243]:
categorical_cols

['department', 'region', 'gender', 'recruitment_channel']

In [244]:
numeric_cols

['no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score']

Viewing the statistics for the numeric columns.

In [245]:
train_inputs[numeric_cols].describe()

,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
count,32884.000000,32884.000000,30386.000000,32884.000000,32884.000000,32884.000000,32884.000000
mean,1.257846,34.811550,3.329165,5.872217,0.351934,0.023872,63.325052
std,0.616027,7.702377,1.258490,4.287976,0.477581,0.152652,13.344067
min,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,40.000000
25%,1.000000,29.000000,3.000000,3.000000,0.000000,0.000000,51.000000
50%,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000
75%,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,75.000000
max,10.000000,60.000000,5.000000,37.000000,1.000000,1.000000,99.000000




Checking the number of categories in each of the categorical columns.

In [246]:
train_inputs[categorical_cols].nunique() # ie how many unique categories exist for each column with categorical columns. is it  string column or a categorical column?

department              9
region                 34
gender                  2
recruitment_channel     3
dtype: int64

region has 39 unique entries, followed by department with 9 unique values.

## Imputing Missing Numeric Data
Replacing the missing values in the 'previous_year_rating' column with the average value in the column using the `SimpleImputer` class from `sklearn.impute`.

In [247]:
#?SimpleImputer  # help for imputer. also gives you the parameters that are used by the imputer.

In [248]:
# create an imputer object that takes some parameters in this case mean
imputer = SimpleImputer(strategy = 'mean')  

Checking the no. of missing values in each numeric column.

In [249]:

hr_df[hr_df.previous_year_rating.isnull()].count()


employee_id             4124
department              4124
region                  4124
gender                  4124
recruitment_channel     4124
no_of_trainings         4124
age                     4124
previous_year_rating       0
length_of_service       4124
KPIs_met >80%           4124
awards_won?             4124
avg_training_score      4124
is_promoted             4124
dtype: int64

These missing values are also spread across the training, test and validation sets. 

In [250]:
train_inputs[train_inputs.previous_year_rating.isna()].count()

department              2498
region                  2498
gender                  2498
recruitment_channel     2498
no_of_trainings         2498
age                     2498
previous_year_rating       0
length_of_service       2498
KPIs_met >80%           2498
awards_won?             2498
avg_training_score      2498
dtype: int64

`fitting` the imputer to the data to compute the mean for each column in the dataset. 

In [251]:
imputer.fit(hr_df[numeric_cols])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [252]:
print(numeric_cols)

['no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score']


In [253]:
list(imputer.statistics_) 
# eg for the numeric cols above, each of the below is the numeric avg. 
#no_of_trainings column average is 1.253010509414684  ,  age column average is  34.80391548679025 etc

[1.253010509414684,
 34.80391548679025,
 3.329255780917055,
 5.865512333965844,
 0.35197416435556855,
 0.023171799737264635,
 63.38675010947307]

In [254]:
#Dsiplay the blank rows in train_inputs under 'previous_year_rating'column
null_train_inputs= train_inputs[train_inputs.previous_year_rating.isnull()]
null_train_inputs


,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
38395,Operations,region_26,f,sourcing,1,27,NaN,1,0,0,61
6377,HR,region_2,m,referred,1,22,NaN,1,1,0,48
20313,Operations,region_2,m,other,1,23,NaN,1,0,0,60
21988,Operations,region_15,m,sourcing,2,31,NaN,1,1,0,56
25218,Technology,region_13,m,sourcing,1,29,NaN,1,0,0,79
...,...,...,...,...,...,...,...,...,...,...,...
24865,Sales & Marketing,region_13,m,sourcing,1,23,NaN,1,1,0,53
29465,Technology,region_7,m,sourcing,1,24,NaN,1,0,0,82
6479,Sales & Marketing,region_31,m,other,1,24,NaN,1,0,0,48
22318,R&D,region_22,m,other,1,32,NaN,1,0,0,87


The missing 'previous_year_rating' values in the training, test and validation datasets can now be filled in using the `transform` method of `imputer`.

 syntax: imputer.transform(train_inputs[numeric_cols]) 


In [255]:
train_inputs[numeric_cols] = imputer.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = imputer.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = imputer.transform(test_inputs[numeric_cols])

In [256]:
# Checking if the missing values we'd seen earlier have been replaced
train_inputs[numeric_cols].isna().sum() 

no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64

The missing values in the 'previous_year-rating' column are now filled in with the mean of that column.

## Scaling Numeric Features

Scaling numeric features to ensure that no particular feature has a disproportionate impact on the model. 
The numeric columns in our dataset have varying ranges and will scaled to a small range of values between $(0,1)$ 


In [257]:
hr_df[numeric_cols].describe()

,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
count,54808.000000,54808.000000,50684.000000,54808.000000,54808.000000,54808.000000,54808.000000
mean,1.253011,34.803915,3.329256,5.865512,0.351974,0.023172,63.386750
std,0.609264,7.660169,1.259993,4.265094,0.477590,0.150450,13.371559
min,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,39.000000
25%,1.000000,29.000000,3.000000,3.000000,0.000000,0.000000,51.000000
50%,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000
75%,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,76.000000
max,10.000000,60.000000,5.000000,37.000000,1.000000,1.000000,99.000000


Scaling using the `MinMaxScaler` from `sklearn.preprocessing` to scale values to the $(0,1)$ range.

In [258]:
scaler = MinMaxScaler()

'Fitting' the scaler to the data by computing the range of values for each numeric column.

In [259]:
scaler.fit(hr_df[numeric_cols])

MinMaxScaler(copy=True, feature_range=(0, 1))

Checking the minimum and maximum values in each column.

In [260]:
print('Minimum:')
list(scaler.data_min_)

Minimum:


[1.0, 20.0, 1.0, 1.0, 0.0, 0.0, 39.0]

In [261]:
print('Maximum:')
list(scaler.data_max_)

Maximum:


[10.0, 60.0, 5.0, 37.0, 1.0, 1.0, 99.0]

Scaling the training, validation and test datasets using the `transform` method of `scaler`.

In [262]:
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

Checking that values in each column lie in the range $(0,1)$

In [263]:
train_inputs[numeric_cols].describe()

,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
count,32884.000000,32884.000000,32884.000000,32884.000000,32884.000000,32884.000000,32884.000000
mean,0.028650,0.370289,0.582293,0.135339,0.351934,0.023872,0.405418
std,0.068447,0.192559,0.302436,0.119110,0.477581,0.152652,0.222401
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667
25%,0.000000,0.225000,0.500000,0.055556,0.000000,0.000000,0.200000
50%,0.000000,0.325000,0.500000,0.111111,0.000000,0.000000,0.350000
75%,0.000000,0.475000,0.750000,0.166667,1.000000,0.000000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Encoding Categorical Data

Converting the categorical data to numbers using one-hot encoding for the categorical columns.

One hot encoding will add a new binary (0/1) column for each unique category of a categorical column. 

In [264]:
hr_df[categorical_cols].nunique()

department              9
region                 34
gender                  2
recruitment_channel     3
dtype: int64

performing one hot encoding using the `OneHotEncoder` class from `sklearn.preprocessing`.

`Fitting` the encoder to the data i.e. identify the full list of categories across all categorical columns.

In [265]:
encoder.fit(hr_df[categorical_cols])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=False)

In [266]:
encoder.categories_

[array(['Analytics', 'Finance', 'HR', 'Legal', 'Operations', 'Procurement',
        'R&D', 'Sales & Marketing', 'Technology'], dtype=object),
 array(['region_1', 'region_10', 'region_11', 'region_12', 'region_13',
        'region_14', 'region_15', 'region_16', 'region_17', 'region_18',
        'region_19', 'region_2', 'region_20', 'region_21', 'region_22',
        'region_23', 'region_24', 'region_25', 'region_26', 'region_27',
        'region_28', 'region_29', 'region_3', 'region_30', 'region_31',
        'region_32', 'region_33', 'region_34', 'region_4', 'region_5',
        'region_6', 'region_7', 'region_8', 'region_9'], dtype=object),
 array(['f', 'm'], dtype=object),
 array(['other', 'referred', 'sourcing'], dtype=object)]

In [267]:
encoded_cols = list(encoder.get_feature_names(categorical_cols))
print(encoded_cols)

['department_Analytics', 'department_Finance', 'department_HR', 'department_Legal', 'department_Operations', 'department_Procurement', 'department_R&D', 'department_Sales & Marketing', 'department_Technology', 'region_region_1', 'region_region_10', 'region_region_11', 'region_region_12', 'region_region_13', 'region_region_14', 'region_region_15', 'region_region_16', 'region_region_17', 'region_region_18', 'region_region_19', 'region_region_2', 'region_region_20', 'region_region_21', 'region_region_22', 'region_region_23', 'region_region_24', 'region_region_25', 'region_region_26', 'region_region_27', 'region_region_28', 'region_region_29', 'region_region_3', 'region_region_30', 'region_region_31', 'region_region_32', 'region_region_33', 'region_region_34', 'region_region_4', 'region_region_5', 'region_region_6', 'region_region_7', 'region_region_8', 'region_region_9', 'gender_f', 'gender_m', 'recruitment_channel_other', 'recruitment_channel_referred', 'recruitment_channel_sourcing']


The encoder has created a list of categories for each of the categorical columns in the dataset. 



All of the above columns will be added to `train_inputs`, `val_inputs` and `test_inputs`.

To perform the encoding, we use the `transform` method of `encoder`.

In [268]:
train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])



We can verify that these new columns have been added to our training, test and validation sets.

In [269]:
#pd.set_option('display.max_columns', None)

In [270]:
test_inputs

,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,department_Analytics,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_1,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
40907,HR,region_16,m,other,0.000000,0.375,1.00,0.250000,0.0,0.0,0.133333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
51939,Sales & Marketing,region_7,m,other,0.222222,0.325,0.00,0.083333,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6978,Finance,region_2,m,other,0.111111,0.450,0.50,0.277778,0.0,0.0,0.416667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
12814,Operations,region_22,f,other,0.000000,0.325,0.00,0.194444,0.0,0.0,0.283333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
22921,Technology,region_7,f,other,0.000000,0.425,0.00,0.083333,0.0,0.0,0.683333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,Analytics,region_7,m,other,0.000000,0.375,0.50,0.027778,0.0,0.0,0.733333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
26056,Sales & Marketing,region_2,m,sourcing,0.111111,0.750,1.00,0.055556,1.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
29981,Sales & Marketing,region_23,m,other,0.000000,0.475,1.00,0.083333,1.0,0.0,0.183333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
44352,Sales & Marketing,region_7,f,other,0.000000,0.275,0.25,0.111111,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [271]:
print('train_inputs:', train_inputs.shape)
print('train_targets:', train_targets.shape)
print('val_inputs:', val_inputs.shape)
print('val_targets:', val_targets.shape)
print('test_inputs:', test_inputs.shape)
print('test_targets:', test_targets.shape)

train_inputs: (32884, 59)
train_targets: (32884,)
val_inputs: (10962, 59)
val_targets: (10962,)
test_inputs: (10962, 59)
test_targets: (10962,)


In [272]:
val_inputs

,department,region,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,department_Analytics,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_1,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
32148,Legal,region_32,m,other,0.000000,0.275,0.582314,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
33302,Procurement,region_22,m,sourcing,0.000000,0.225,0.500000,0.055556,0.0,0.0,0.516667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
38582,Technology,region_13,m,sourcing,0.000000,0.300,0.500000,0.138889,1.0,0.0,0.716667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26817,Sales & Marketing,region_22,m,other,0.333333,0.175,0.582314,0.000000,0.0,0.0,0.216667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
50418,Finance,region_4,m,other,0.000000,0.375,0.750000,0.111111,0.0,0.0,0.366667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46589,Technology,region_26,m,other,0.000000,0.200,0.000000,0.083333,0.0,0.0,0.683333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
42762,Analytics,region_31,m,other,0.000000,0.225,0.500000,0.138889,0.0,0.0,0.816667,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
8200,Sales & Marketing,region_2,f,other,0.000000,0.900,0.500000,0.388889,0.0,0.0,0.183333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1391,Operations,region_2,f,sourcing,0.000000,0.400,0.750000,0.027778,1.0,0.0,0.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [273]:
val_targets

32148    0
33302    0
38582    0
26817    0
50418    0
        ..
46589    0
42762    0
8200     0
1391     0
43269    0
Name: is_promoted, Length: 10962, dtype: int64

## Training the Logistic Regression Model to predict the 'is_promoted' values.

Logistic regression is the technique chosen to solve this binary classification problem

To train a logistic regression model, we use the `LogisticRegression` class from Scikit-learn.

In [274]:
model = LogisticRegression(solver='liblinear')

Training the model using `model.fit`.

In [275]:
model.fit(train_inputs[numeric_cols + encoded_cols], train_targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

## Predictions and Evaluating the Model

Using the trained model to make predictions on the training and test datasets 

In [276]:
X_train = train_inputs[numeric_cols + encoded_cols]
X_val = val_inputs[numeric_cols + encoded_cols]
X_test = test_inputs[numeric_cols + encoded_cols]

In [277]:
train_preds = model.predict(X_train)

In [278]:
train_preds

array([0, 0, 0, ..., 0, 0, 0])

In [279]:
train_targets

50563    0
31423    0
50189    1
23603    0
25759    0
        ..
48824    0
50281    0
6587     0
51986    0
24101    0
Name: is_promoted, Length: 32884, dtype: int64

Testing the accuracy of the model's predictions by computing the percentage of matching values in `train_preds` and `train_targets` using the `accuracy_score` function from `sklearn.metrics`.

In [280]:
accuracy_score(train_targets, train_preds)

0.9304829096217005

The model achieves an accuracy of 93.04% on the training set. 

Visualizing a breakdown of correctly and incorrectly classified inputs using a confusion matrix.



In [281]:
confusion_matrix(train_targets, train_preds, normalize='true')

array([[0.99640611, 0.00359389],
       [0.76879633, 0.23120367]])

In [282]:
print("Confusion Matrix : \n",confusion_matrix(train_targets, train_preds))

Confusion Matrix : 
 [[29943   108]
 [ 2178   655]]


In [283]:
print(' f1 score: ',f1_score(val_targets, val_preds)*100,'\n')
print(' Accuracy: ',accuracy_score(val_targets, val_preds)*100,'\n')
print(' precision score: ',precision_score(val_targets, val_preds)*100,'\n')
print(' recall score: ',recall_score(val_targets, val_preds)*100,'\n')
print(" Classification report: \n",classification_report(val_targets, val_preds))

 f1 score:  36.734693877551024 

 Accuracy:  93.21291735084839 

 precision score:  86.74698795180723 

 recall score:  23.300970873786408 

 Classification report: 
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     10035
           1       0.87      0.23      0.37       927

    accuracy                           0.93     10962
   macro avg       0.90      0.61      0.67     10962
weighted avg       0.93      0.93      0.91     10962



Computing the model's accuracy on the validation and test datasets

In [284]:
accuracy_score(val_targets, val_preds)


0.9321291735084839

In [285]:
accuracy_score(test_targets, test_preds)

0.9323116219667944

The accuracy of the model on the test and validation set are above 93.04% which suggests that our model generalizes well to data it hasn't seen before. 

